In [5]:
import numpy as np
import torch
from collections import namedtuple
import gzip
import pickle
from pathlib import Path
import torch
from model import SessionGraph, forward, trans_to_cuda, trans_to_cpu
from utils import Data

In [6]:
def get_bigram_matrix(filename_tagnn_input_data = '../YooChoose 1_64 Dataset Preparation/ziob_yoochoose_1_64n.pcklz', number_of_nodes = -1, batch_size = 32, training_epochs = 50):
    with gzip.open(filename_tagnn_input_data, 'rb') as f:
        train_data = pickle.load(f)
        test_data = pickle.load(f)
        pids = pickle.load(f)

    if number_of_nodes == -1:
        number_of_nodes = len(pids)
    print(number_of_nodes)
    vertex_ranking = np.zeros((number_of_nodes))
    train_data = Data(train_data, shuffle=True)
    c = 0
    e = 0
    print(f"len(slices): {len(train_data.generate_batch(batch_size))}, no : {number_of_nodes}")
    slices = train_data.generate_batch(batch_size)
    for i, j in zip(slices, np.arange(len(slices))):
        if c % 100 == 0 and c != 0:
            print(f"{c}/{len(slices)}")
        alias_inputs, A, items, mask, targets = train_data.get_slice(i)
        alias_inputs = trans_to_cuda(torch.Tensor(alias_inputs).long())
        items = trans_to_cuda(torch.Tensor(items).long())
        A = trans_to_cuda(torch.Tensor(np.array(A)).float())
        mask = trans_to_cuda(torch.Tensor(mask).long())
        for k in range(batch_size):
            if k > mask.shape[0] - 1:
                break
            for l in range(mask[k].sum().item()):
                    try:
                        vertex_ranking[items[k][alias_inputs[k]][l].item()] += 1
                    except:
                        print(f"k: {k}, l: {l}")
                        e += 1
        c += 1
    print(f"errors: {e}")
    return vertex_ranking

In [7]:
vertex_ranking = get_bigram_matrix()

16191
len(slices): 11520, no : 16191
100/11520
200/11520
300/11520
400/11520
500/11520
600/11520
700/11520
800/11520
900/11520
1000/11520
1100/11520
1200/11520
1300/11520
1400/11520
1500/11520
1600/11520
1700/11520
1800/11520
1900/11520
2000/11520
2100/11520
2200/11520
2300/11520
2400/11520
2500/11520
2600/11520
2700/11520
2800/11520
2900/11520
3000/11520
3100/11520
3200/11520
3300/11520
3400/11520
3500/11520
3600/11520
3700/11520
3800/11520
3900/11520
4000/11520
4100/11520
4200/11520
4300/11520
4400/11520
4500/11520
4600/11520
4700/11520
4800/11520
4900/11520
5000/11520
5100/11520
5200/11520
5300/11520
5400/11520
5500/11520
5600/11520
5700/11520
5800/11520
5900/11520
6000/11520
6100/11520
6200/11520
6300/11520
6400/11520
6500/11520
6600/11520
6700/11520
6800/11520
6900/11520
7000/11520
7100/11520
7200/11520
7300/11520
7400/11520
7500/11520
7600/11520
7700/11520
7800/11520
7900/11520
8000/11520
8100/11520
8200/11520
8300/11520
8400/11520
8500/11520
8600/11520
8700/11520
8800/11520
8900

In [8]:
sorted_ranking, indices = torch.sort(trans_to_cuda(torch.Tensor(vertex_ranking)), descending=True)

In [9]:
sorted_ranking.shape

torch.Size([16191])

In [10]:
sorted_ranking

tensor([15593., 14333., 14253.,  ...,     0.,     0.,     0.], device='cuda:0')

In [ ]:
merged_tensor = torch.stack((sorted_ranking, indices), dim=1)
torch.save(merged_tensor, 'vertex_ranking.pt')